In [1]:
import pandas as pd
import fasttext
import numpy as np
import csv

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

import string

import warnings
warnings.filterwarnings('ignore')

In [2]:
# data = pd.read_excel('tagging.xlsx')
# df = data[['label', 'questions']]
# df.head()

# Writing csv to txt file
# df.to_csv('abc.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [3]:
# Setting up the stopwords dictionary
stopwords = [
             'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
             'ourselves', 'you', "you're", "you've", "you'll", 
             "you'd", 'your', 'yours', 'yourself', 'yourselves', 
             'he', 'him', 'his', 'himself', 'she', "she's", 'her', 
             'hers', 'herself', 'it', "it's", 'its', 'itself', 
             'they', 'them', 'their', 'theirs', 'themselves', 'what', 
             'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 
             'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 
             'being', 'have', 'has', 'had', 'having', 'do', 'does', 
             'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 
             'or', 'because', 'as', 'until', 'while', 'of', 'at', 
             'by', 'for', 'with', 'about', 'against', 'between', 
             'into', 'through', 'during', 'before', 'after', 'above', 
             'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
             'off', 'over', 'under', 'again', 'further', 'then', 'once', 
             'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
             'both', 'each', 'few', 'more', 'most', 'other', 'some', 
             'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
             'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
             'don', "don't", 'should', "should've", 'now', 'd', 'll', 
             'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 
             "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
             "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
             'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 
             'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't", 'fuck', 'fucker',
             'lah', 'la', 'leh', 'lor', 'nah', 'ya', 'yah', 'shit', 'ass', 'asshole',
             'le', 'already', 'liao', 'liaoz', 'u', 'cheebye', 'lanjiao',
             'nabei', 'kaopei', 'knnb', 'cb', 'cheebye', 'fucked', 'fucks',
             'bitch', 'bitches', 'scumbag', 'fuckface', 'wtf', 'ffs', 'siao',
             'walao', 'waliao', 'ttyl', 'orhhhh', 'sai'
             ]

In [4]:
# Setting up nltk's lemmatizer
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [5]:
f = open("fasttext_input.txt", "r", encoding='utf-8')
# Converting to lower case
# f = f.read().lower()
# print(f.read())

In [6]:
model = fasttext.train_supervised('fasttext_input.txt', 
                                  epoch=100,
                                  lr=0.1,
                                  wordNgrams=2,
                                  loss='softmax',
                                  ws=10
                                 )

# Model v1.0
# By right need a test set for this, then tune with test set
# But that's for the future

# model.words

In [7]:
model.predict('what is my name', k=1)[0][0]

'__label__aprogress'

In [8]:
qasmall = pd.read_excel('tagging.xlsx')
qabig = pd.read_excel('q&adatabaseexpanded.xlsx')

In [9]:
qasmall['prompt'] = qasmall['prompt'].map(lambda x: x.lower())
qasmall.head()

,label,prompt,response
0,__label__adminpayment,i have settled my payment with my consultant.,Thank you. We will inform our Finance and your...
1,__label__adminpayment,i have made my payment with my consultant.,Thank you. We will inform our Finance and your...
2,__label__adminpayment,when do i have to make payment?,You have to make the payment before your 1st F...
3,__label__adminpayment,how do i make payment?,Payment can be made via Giro/ TT/ Cheque/ Payn...
4,__label__adminpayment,is there any late payment fee?,"No. There is no late payment fee. However, you..."


In [10]:
qabig['prompt'] = qabig['prompt'].map(lambda x: x.lower())
qabig.head()

,label,prompt,response
0,__label__adminpayment,i have settled my payment with my consultant.,Thank you. We will inform our Finance and your...
1,__label__adminpayment,i have made my payment with my consultant.,Thank you. We will inform our Finance and your...
2,__label__adminpayment,i paid my course fees already.,Thank you. We will inform our Finance and your...
3,__label__adminpayment,i have paid my course fees.,Thank you. We will inform our Finance and your...
4,__label__adminpayment,my course fees have been paid.,Thank you. We will inform our Finance and your...


In [11]:
def fasttext_output(user_input):
    return model.predict(user_input, k=1)[0][0]

In [ ]:
flag = True
print('Hello, please ask me a question!')
while(flag == True):
    user_input = input()
    user_input = user_input.lower()
    bot_response = ''
    
    # Fasttext model to classify user question into different categories
    matched_label = fasttext_output(user_input)
    # Mask/filter for dataframe
    mask = qabig.label == matched_label
    
    # Temporary string to the prompts from the matched label
    prompts_corpus = ''
    for i in qabig[mask]['prompt'].values:
        prompts_corpus += i
    prompts_corpus = prompts_corpus.replace('.', '. ').replace('?', '? ').replace('!', '! ')
    
    # Tokenizing the prompts corpus into sentence tokens
    # Adding the user's input into the list of sentence tokens
    prompts_sent_token = nltk.sent_tokenize(prompts_corpus)
    prompts_sent_token.append(user_input)

    # Initializing the Tfidf Vecotizer
    # Using LemNormalize function to tokenize it
    # Stopwords taken from the manually defined list
    # ngrams set to 1 and 2
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, 
                               stop_words=stopwords, 
                               ngram_range=(1,2))
    
    # Fitting the tfidf model with the sentence tokens
    tfidf = TfidfVec.fit_transform(prompts_sent_token) 
    cosine_vals = cosine_similarity(tfidf[-1], tfidf)
    index = cosine_vals.argsort()[0][-2]
    flat = cosine_vals.flatten()
    flat.sort()
    matched_cosine = flat[-2]
    
    if matched_cosine == 0:
        bot_response = bot_response + "I'm sorry, I do not understand you."
    
    else:
        try:
            # bot_response = bot_response + qabig[(qabig['prompt'] == prompts_sent_token[index])]['resp'][qabig[qabig['prompt'] == prompts_sent_token[index]]['resp'].index[0]]
            # bot_response = bot_response + qabig[(qabig['prompt'].str.contains(prompts_sent_token[index]))]['resp'][qabig[qabig['prompt'] == prompts_sent_token[index]]['resp'].index[0]]
            bot_response = bot_response + qabig[qabig['prompt'] == prompts_sent_token[index]]['response'].values[0]
        except:
            bot_response = bot_response + qabig[(qabig['prompt'].str.contains(prompts_sent_token[index]))]
    prompts_sent_token.remove(user_input)
    
    print('Question: ', user_input)
    print('Bot: ', bot_response)
    print('-------------------------------------------------------------')
    print('Matched Label: ', matched_label)
    print('Matched Label Score: ', model.predict(user_input, k=1)[1])
    print('Matched Question: ', prompts_sent_token[index])
    print('2nd Best Match: ', prompts_sent_token[index-1])
    print('3rd Best Match: ', prompts_sent_token[index-2])
#    print('Cosine Similarity Values: ', flat)
    print('*************************************************************')
#    print(tfidf)
#    print(tfidf[-1])

Hello, please ask me a question!
Hello
Question:  hello
Bot:  I'm sorry, I do not understand you.
-------------------------------------------------------------
Matched Label:  __label__admingeneral
Matched Label Score:  [0.93318278]
Matched Question:  yes i need assistance.
2nd Best Match:  i need you to guide me.
3rd Best Match:  i need some assistance.
*************************************************************
